In [1]:
import pandas as pd
import numpy as np
from pandas_gbq import gbq

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Для Линии Тренда
from numpy import arange,array,ones
from scipy import stats

In [2]:
from IPython.display import display,HTML
HTML('''<button style="align: center; margin:0 auto; display: block; background-color: #4CAF50;  color: white;  border: 2px solid #4CAF50; /* Green */" onclick="jQuery('.code_cell .input_area').toggle();
    jQuery('.prompt').toggle();"><text style = background-color: #4CAF50;>Нажми меня</text></button>''')

In [25]:
#Входные данные

#  Если эксперимент на RU - Нужно поменять 
#  `movavi---owox-demo.969794
#  На 
#  `movavi---owox-demo.8254485

#даты эксперимента
date1 = '2020-06-01'
date2 = '2020-06-30'

#-------------------------------------------------------------------------------------------------------

query_config = {
    'query': {
        'parameterMode': 'NAMED',
        'queryParameters': [
            {
                'name': 'date1',
                'parameterType': {'type': 'STRING'},
                'parameterValue': {'value': date1}
            },
            {
                'name': 'date2',
                'parameterType': {'type': 'STRING'},
                'parameterValue': {'value': date2}
            }
        ]
    }
} 

In [26]:
query_main = """


With ga_table as 
(
    SELECT distinct 
    fullVisitorId                           as user                        --ID Юзера
    ,PARSE_DATE("%Y%m%d", date)             as date                        --Дата в формате ГГГГ-ММ-ДД, а не в формате ГГГГММДД
    ,visitStartTime + hits.time/1000        as hit_time                    --Временная отметка хита в секундах
    ,LOWER(hits.page.pagePath)              as page                        --Адрес страницы
    ,hits.time                               as hit_time_1
    ,LEAD(hits.time, 1) OVER(PARTITION BY fullVisitorId, visitStartTime ORDER BY (visitStartTime + (hits.time / 1000)))  as hit_time_2   --Следующий хит тайм для этого юзера (нужно для определения среднего времени на странице)

    ,channelGrouping                        as channel
    ,geoNetwork.country                     as country

    ,hits.isExit                            as exit
    ,hits.isEntrance                        as entrance

    FROM
    `movavi---owox-demo.969794.ga_sessions_*`,     --Формат ГГГГММДД
    UNNEST(hits) AS hits
        
    WHERE  
    _TABLE_SUFFIX BETWEEN FORMAT_DATE("%Y%m%d", DATE(@date1)) AND FORMAT_DATE("%Y%m%d", DATE(@date2))          			--Даты эксперимента
    Order by fullVisitorId, hit_time
)


, ga_table_joined as 
(

    SELECT distinct 
    user_1      as ProductPage
    ,date_1     as date
    ,channel
    ,country
    ,local
    ,exit       as exit
    ,entrance   as entrance
    ,bounce
    ,menu       as menu
    ,PPC
    ,SEO
    ,Direct
    ,time_on_page
    
    ,user_2     as academic_for_schools
    ,user_3     as buy_academic
    ,user_4     as buy_academic_form
    ,user_5     as receive_offer
    ,user_6     as receive_offer_form
    ,user_7     as receive_offer_all
    ,user_8     as receive_offer_all_form
    ,user_9     as academic_remote_offer
    ,user_10    as academic_remote_offer_form
    

    FROM
    (
        SELECT *
        FROM

        (
----------------------------------------------------Продуктовая
            SELECT 
            user                        as user_1
            ,CASE
  WHEN LENGTH(CAST((EXTRACT(month FROM date)) AS STRING)) = 1
  THEN CONCAT((CAST((EXTRACT(year FROM date)) AS STRING)),'-0',(CAST((EXTRACT(month FROM date)) AS STRING)))
  ELSE CONCAT((CAST((EXTRACT(year FROM date)) AS STRING)),'-',(CAST((EXTRACT(month FROM date)) AS STRING)))
END as date_1
            ,hit_time
            ,page

            ,channel
            ,country
            ,CASE WHEN exit = TRUE THEN user ELSE NULL END as exit
            ,CASE WHEN entrance = TRUE THEN user ELSE NULL END as entrance
            ,CASE WHEN entrance = TRUE AND exit = TRUE THEN user ELSE NULL END as bounce
            ,CASE WHEN REGEXP_CONTAINS(LOWER(page), 'main_menu') THEN user ELSE NULL END as menu
            
            ,CASE WHEN channel = "Paid Search" THEN user ELSE NULL END as PPC
            ,CASE WHEN channel = "Organic Search" THEN user ELSE NULL END as SEO
            ,CASE WHEN channel = "Direct" THEN user ELSE NULL END as Direct
            
             --Считаем время на странице до след хита. Если это время больше 30-ти минут или меньше 6-ти секунд, то пишем NULL
            ,CASE 
              WHEN (Round((hit_time_2 - hit_time_1)/60000,2) <30) AND (Round((hit_time_2 - hit_time_1)/60000,2) > 0.1)     
              THEN Round((hit_time_2 - hit_time_1)/60000,2)
            ELSE NULL
            END AS time_on_page
            ,CASE 
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/de") THEN 'DE'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/fr") THEN 'FR'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/it") THEN 'IT'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/es") THEN 'ES'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/pt") THEN 'PT'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/nl") THEN 'NL'
                --WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/cz") THEN 'CZ'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/dk") THEN 'DK'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/no") THEN 'NO'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/se") THEN 'SE'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/tr") THEN 'TR'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/pl") THEN 'PL'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/jp") THEN 'JP'
                --WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/ar") THEN 'AR'
                --WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/cn") THEN 'CN'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/kr") THEN 'KR'
                WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/zh") THEN 'ZH'
                --WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/hu") THEN 'HU'
                --WHEN REGEXP_CONTAINS(page, r"^edu.movavi.com/th") THEN 'TH'
            ELSE 'EN'
            END AS local
            
            
            

            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(LOWER(page), 'edu.movavi.com/index')
                    OR
                REGEXP_CONTAINS(LOWER(page), r'edu.movavi.com/([a-z]{2})$')
                
            -- Да, вот такой вот академик нестандартный, что локаль указывает в конце урла (не только на главной) и без индекса.     
                
        ) as Step_1
        
----------------------------------------------------academic-for-schools--------------Переключлка в меню
        LEFT JOIN
        (
            SELECT 
            user                          as user_2
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/academic-for-schools') 
        ) as Step_2
        ON
        Step_1.user_1 = Step_2.user_2 AND Step_1.hit_time <= Step_2.hit_time 
        
----------------------------------------------------/buy-academic--------------первые 3 BuyNow кнопки        
        LEFT JOIN
        (
            SELECT 
            user                          as user_3
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/buy-academic') 
        ) as Step_3
        ON
        Step_3.user_3 = Step_1.user_1 AND Step_1.hit_time <= Step_3.hit_time  
        
----------------------------------------Заполнение формы с------------/buy-academic--------------первые 3 BuyNow кнопки        
        LEFT JOIN
        (
            SELECT 
            user                          as user_4
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form117963270/submitted')       --EN
                OR REGEXP_CONTAINS(page, '/tilda/form164174618/submitted')    --FR 
                OR REGEXP_CONTAINS(page, '/tilda/form164185291/submitted')    --DE 
                OR REGEXP_CONTAINS(page, '/tilda/form164216118/submitted')    --ES
                OR REGEXP_CONTAINS(page, '/tilda/form164254643/submitted')    --IT
                OR REGEXP_CONTAINS(page, '/tilda/form164491814/submitted')    --NL
                OR REGEXP_CONTAINS(page, '/tilda/form165309159/submitted')    --KR
                OR REGEXP_CONTAINS(page, '/tilda/form165940151/submitted')    --PT
                OR REGEXP_CONTAINS(page, '/tilda/form165950908/submitted')    --PL
                OR REGEXP_CONTAINS(page, '/tilda/form166015972/submitted')    --JP
                OR REGEXP_CONTAINS(page, '/tilda/form168224460/submitted')    --NO
                OR REGEXP_CONTAINS(page, '/tilda/form168239460/submitted')    --DK
                OR REGEXP_CONTAINS(page, '/tilda/form168259869/submitted')    --SE
                OR REGEXP_CONTAINS(page, '/tilda/form166279473/submitted')    --ZH
                OR REGEXP_CONTAINS(page, '/tilda/form166310008/submitted')    --TR
                
                
                
                
        ) as Step_4
        ON
        Step_4.user_4 = Step_3.user_3 AND Step_3.hit_time <= Step_4.hit_time  
        
----------------------------------------------------/receive-offer--------------4-5 BuyNow кнопки
        LEFT JOIN
        (
            SELECT 
            user                          as user_5
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/receive-offer') AND NOT
                REGEXP_CONTAINS(page, 'edu.movavi.com/receive-offer-') 
        ) as Step_5
        ON
        Step_5.user_5 = Step_1.user_1 AND Step_1.hit_time <= Step_5.hit_time 
        
----------------------------------------Заполнение формы с------------/receive-offer--------------4-5 BuyNow кнопки
        LEFT JOIN
        (
            SELECT 
            user                          as user_6
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form110272193/submitted')     --EN
                OR REGEXP_CONTAINS(page, '/tilda/form163959630/submitted')  --FR
                OR REGEXP_CONTAINS(page, '/tilda/form164181765/submitted')  --DE
                OR REGEXP_CONTAINS(page, '/tilda/form164208890/submitted')  --ES
                OR REGEXP_CONTAINS(page, '/tilda/form164247265/submitted')  --IT
                OR REGEXP_CONTAINS(page, '/tilda/form164486549/submitted')  --NL
                OR REGEXP_CONTAINS(page, '/tilda/form165307104/submitted')  --KR
                OR REGEXP_CONTAINS(page, '/tilda/form165398487/submitted')  --PT
                OR REGEXP_CONTAINS(page, '/tilda/form165945713/submitted')  --PL
                OR REGEXP_CONTAINS(page, '/tilda/form166009304/submitted')  --JP
                OR REGEXP_CONTAINS(page, '/tilda/form168210280/submitted')  --NO
                OR REGEXP_CONTAINS(page, '/tilda/form168232142/submitted')  --DK
                OR REGEXP_CONTAINS(page, '/tilda/form168244460/submitted')  --SE
                OR REGEXP_CONTAINS(page, '/tilda/form166277810/submitted')  --ZH
                OR REGEXP_CONTAINS(page, '/tilda/form166294962/submitted')  --TR
                
        ) as Step_6
        ON
        Step_6.user_6 = Step_5.user_5 AND Step_5.hit_time <= Step_6.hit_time 
        
----------------------------------------------------/receive-offer-all--------------6 BuyNow кнопка
        LEFT JOIN
        (
            SELECT 
            user                          as user_7
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/receive-offer-all') 
        ) as Step_7
        ON
        Step_7.user_7 = Step_1.user_1 AND Step_1.hit_time <= Step_7.hit_time 

----------------------------------------Заполнение формы с------------/receive-offer-all--------------6 BuyNow кнопка
        LEFT JOIN
        (
            SELECT 
            user                          as user_8
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form169017050/submitted')     --EN
                OR REGEXP_CONTAINS(page, '/tilda/form169304721/submitted')  --FR
                OR REGEXP_CONTAINS(page, '/tilda/form169310398/submitted')  --DE
                OR REGEXP_CONTAINS(page, '/tilda/form169341999/submitted')  --ES
                OR REGEXP_CONTAINS(page, '/tilda/form169343856/submitted')  --IT
                OR REGEXP_CONTAINS(page, '/tilda/form169346788/submitted')  --NL
                OR REGEXP_CONTAINS(page, '/tilda/form169350314/submitted')  --KR
                OR REGEXP_CONTAINS(page, '/tilda/form171734520/submitted')  --PT
                OR REGEXP_CONTAINS(page, '/tilda/form171736490/submitted')  --PL
                OR REGEXP_CONTAINS(page, '/tilda/form171736836/submitted')  --JP
                OR REGEXP_CONTAINS(page, '/tilda/form171737240/submitted')  --NO
                OR REGEXP_CONTAINS(page, '/tilda/form171737485/submitted')  --DK
                OR REGEXP_CONTAINS(page, '/tilda/form171738206/submitted')  --SE
                OR REGEXP_CONTAINS(page, '/tilda/form171738590/submitted')  --ZH
                OR REGEXP_CONTAINS(page, '/tilda/form171739142/submitted')  --TR 
        ) as Step_8
        ON
        Step_8.user_8 = Step_7.user_7 AND Step_7.hit_time <= Step_8.hit_time 

----------------------------------------------------/academic-remote-offer--------------Поп-ап
        LEFT JOIN
        (
            SELECT 
            user                          as user_9
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, 'edu.movavi.com/academic-remote-offer') 
        ) as Step_9
        ON
        Step_9.user_9 = Step_1.user_1 AND Step_1.hit_time <= Step_9.hit_time 

----------------------------------------Заполнение формы с------------/academic-remote-offer--------------Поп-ап
        LEFT JOIN
        (
            SELECT 
            user                          as user_10
            ,hit_time
            ,page
            FROM ga_table
            WHERE 
                REGEXP_CONTAINS(page, '/tilda/form170970217/submitted') 
        ) as Step_10
        ON
        Step_10.user_10 = Step_9.user_9 AND Step_9.hit_time <= Step_10.hit_time 
        
    )    

)  




SELECT distinct * 
FROM ga_table_joined
Order by  date desc


"""

In [27]:
df = gbq.read_gbq(query=query_main, dialect='standard',project_id='movavi---owox-demo', configuration=query_config, progress_bar_type=None)

In [28]:
pd.set_option('display.max_rows', 100)

In [29]:
df

,ProductPage,date,channel,country,local,exit,entrance,bounce,menu,PPC,...,time_on_page,academic_for_schools,buy_academic,buy_academic_form,receive_offer,receive_offer_form,receive_offer_all,receive_offer_all_form,academic_remote_offer,academic_remote_offer_form
0,1470585258869881244,2020-06,Paid Search,United States,EN,None,None,None,None,1470585258869881244,...,0.17,None,None,None,None,None,None,None,None,None
1,4220856812267154095,2020-06,Paid Search,Ireland,EN,None,None,None,4220856812267154095,4220856812267154095,...,0.19,None,None,None,None,None,None,None,4220856812267154095,None
2,5014181205057081525,2020-06,Paid Search,Singapore,EN,None,None,None,5014181205057081525,5014181205057081525,...,NaN,None,None,None,None,None,None,None,None,None
3,7350330336734466543,2020-06,Organic Search,United States,EN,None,None,None,7350330336734466543,None,...,4.14,None,None,None,None,None,None,None,None,None
4,7611629627264164653,2020-06,Paid Search,Egypt,EN,None,None,None,7611629627264164653,7611629627264164653,...,0.24,None,None,None,None,None,None,None,7611629627264164653,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23292,8597067731111580574,2020-06,Organic Search,Egypt,EN,None,None,None,8597067731111580574,None,...,0.14,None,None,None,None,None,None,None,None,None
23293,8693914093971193155,2020-06,Organic Search,United States,EN,None,None,None,None,None,...,1.20,None,None,None,None,None,None,None,None,None
23294,8860556888024639905,2020-06,Paid Search,United States,EN,None,None,None,8860556888024639905,8860556888024639905,...,0.42,8860556888024639905,None,None,None,None,None,None,None,None
23295,9020708620697091392,2020-06,Organic Search,South Africa,EN,None,None,None,None,None,...,NaN,None,9020708620697091392,None,9020708620697091392,None,None,None,9020708620697091392,None


In [30]:
df_copy = df.copy()

In [31]:
df = df_copy

## Конверсия с продуктовой в посещения страниц и заполнения форм по локалям
(Вкладка Локали)

In [32]:
df_count = df.groupby(['local']).nunique()
df_count = df_count[['ProductPage', 'academic_for_schools', 'buy_academic', 'buy_academic_form', 'receive_offer', 'receive_offer_form', 'receive_offer_all', 'receive_offer_all_form', 'academic_remote_offer', 'academic_remote_offer_form']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['local'], ascending=True)

,ProductPage,academic_for_schools,buy_academic,buy_academic_form,receive_offer,receive_offer_form,receive_offer_all,receive_offer_all_form,academic_remote_offer,academic_remote_offer_form
local,,,,,,,,,,
DE,27,11,5,2,6,2,2,0,4,0
DK,3,3,3,1,2,1,2,0,2,0
EN,10202,686,1562,367,431,133,49,16,2708,396
ES,62,7,17,5,6,2,2,0,5,1
FR,24,4,8,1,4,2,1,0,3,0
IT,18,9,8,2,9,1,2,0,3,0
JP,3,2,2,1,1,0,1,0,2,0
KR,53,20,22,6,12,0,4,0,2,0
NL,7,1,1,0,1,0,1,0,1,0


## Пока не вставлял эту инфу в док, т.к. без толку, слишком мало юзеров

In [33]:
df_count = df.groupby(['local']).nunique()
df_count = df_count[['ProductPage', 'exit', 'entrance', 'bounce', 'menu']]
#df_count = df_count.assign(Offer_Conversion=round((df_count['Offer'] / df_count['ProductPage']),4))
#df_count = df_count.assign(BuyNow_Conversion=round((df_count['BuyNow'] / df_count['ProductPage']),4))
#df_count = df_count.assign(Other_Conversion=round((df_count['Other_PO'] / df_count['ProductPage']),4))
df_count.sort_values(by=['local'], ascending=True)

,ProductPage,exit,entrance,bounce,menu
local,,,,,
DE,27,5,2,0,0
DK,3,0,0,0,0
EN,10202,3993,2283,1389,7127
ES,62,24,16,5,0
FR,24,9,6,3,0
IT,18,3,2,1,0
JP,3,0,0,0,0
KR,53,15,9,6,0
NL,7,4,0,0,0


In [34]:
df_count = df.groupby(['local']).count()
df_count = df_count[['ProductPage', 'exit', 'entrance', 'bounce', 'menu', 'PPC', 'SEO', 'Direct', 'time_on_page']]
df_count = df_count.assign(Exit_percent=round((df_count['exit'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Entrance_percent=round((df_count['entrance'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Bounce_rate=round((df_count['bounce'] / df_count['entrance']),4)*100)
df_count = df_count.assign(Menu_Percent=round((df_count['menu'] / df_count['ProductPage']),4)*100)

df_count = df_count.assign(PPC_Percent=round((df_count['PPC'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(SEO_Percent=round((df_count['SEO'] / df_count['ProductPage']),4)*100)
df_count = df_count.assign(Direct_Percent=round((df_count['Direct'] / df_count['ProductPage']),4)*100)

df_count = df_count[['ProductPage', 'Exit_percent', 'Entrance_percent', 'Bounce_rate', 'Menu_Percent', 'PPC_Percent', 'SEO_Percent', 'Direct_Percent']]
df_count.sort_values(by=['local'], ascending=True)

,ProductPage,Exit_percent,Entrance_percent,Bounce_rate,Menu_Percent,PPC_Percent,SEO_Percent,Direct_Percent
local,,,,,,,,
DE,55,9.09,3.64,0.00,0.00,7.27,21.82,54.55
DK,29,0.00,0.00,NaN,0.00,0.00,0.00,75.86
EN,22519,19.46,12.07,57.06,60.46,39.26,21.54,28.70
ES,135,20.74,14.07,26.32,0.00,34.07,31.85,17.78
FR,52,19.23,11.54,50.00,0.00,15.38,38.46,30.77
IT,45,8.89,4.44,50.00,0.00,6.67,26.67,37.78
JP,8,0.00,0.00,NaN,0.00,25.00,0.00,25.00
KR,114,14.04,9.65,54.55,0.00,35.96,24.56,14.91
NL,20,20.00,0.00,NaN,0.00,25.00,5.00,25.00
